<a href="https://colab.research.google.com/github/Rishika70/LLM/blob/main/Vectara_Retrieval_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Workaround to avoid following error at notebook
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Install all required libraries
Huggingface libraries to use Mistral 7B LLM (Open Source LLM)

LangChain library to call LLM to generate reposne based on the prompt

In [ ]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# FAISS Vector Databses specific Libraries
!pip install -q -U faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00


In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

import torch

from langchain.llms import HuggingFacePipeline

from langchain.chains import ConversationalRetrievalChain

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [ ]:

origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Load the PDF and create Chunk of texts

In [ ]:

# Load the pdf file
loader = PyPDFLoader('/Introduction-to-Risk-Management-ENGLISH.pdf')
documents = loader.load()

# Split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(documents)

In [ ]:

# HuggingFace embeddings to create embedding of chunked docs to store at
#Vector Database

embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

# Store chunk of pdf files at FAISS vector database by using HuggingFace Embedding model


faiss_db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [ ]:
# Connect query to FAISS index using a retriever
retriever = faiss_db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:

# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

# Prompt Engineering

In [ ]:

#Get the answer of question from Vector Database
import sys

def get_user_input():
    return input('Prompt: ').lower()

def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: What is Risk management?
Answer:  Risk management is the process of identifying, evaluating, and controlling the risks associated with a particular activity or project. It involves identifying potential threats or hazards, assessing their likelihood and potential impact, and developing strategies to mitigate or eliminate them. Risk management is an ongoing process that requires regular monitoring and evaluation to ensure that risks are being effectively managed.



In [ ]:
def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: Types of risk management and their main function
Answer: 

Types of risk management and their main function

There are five primary sources of risk in agricultural production: Production, Marketing, Financial, Legal, and Human. Each of these sources presents unique challenges and requires specific strategies to mitigate them.

Production risk involves variability in agricultural outcomes, such as weather, climate changes, pests, diseases, technology, genetics, machinery efficiency, and the quality of inputs. To manage this risk, farmers can implement measures such as crop rotation, using resistant varieties, investing in efficient machinery, and maintaining good soil health.

Marketing risk involves variability in prices farmers receive for their products or pay for production inputs. To manage this risk, farmers can diversify their markets, negotiate contracts, and monitor market trends.

Financial risk involves threats to the financial health of the business, such as the cost

In [ ]:
def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: Describe Financial and Legal Risks
Answer:  Financial risk refers to the potential negative impact on the financial stability of a business. This can include the cost and availability of capital, the ability to meet cash flow needs, the ability to maintain and grow equity, and the ability to absorb short-term financial shocks. Legal risk, on the other hand, refers to the potential negative impact on a business due to legal issues such as disputes or disagreements between individuals, between individuals and groups, or between groups. This can include contractual arrangements, business organization, laws and regulations, tort liability, and public policies and attitudes.



#Vectra HHEM (Hallucination Evaluation Model)
## Vectara’s industry-leading Relevance (the “R” in RAGaaS), simple and powerful APIs, and offloading many components in the GenAI pipeline to accelerate development are why developers who’ve built their own come back to Vectara. Vectara never trains on your data, allowing businesses to embed generative AI capabilities without the risk of data or privacy violations.

In [ ]:
!pip install vectara

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=83cf6024d27bb8ea3927fa4d30123f5d9efd62cee1a4f40137e4da7931b2a5e6
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.fake import FakeEmbeddings
from langchain_community.vectorstores import Vectara

In [ ]:
import os
import getpass

os.environ["VECTARA_CUSTOMER_ID"] = getpass.getpass("1668875253")
os.environ["VECTARA_CORPUS_ID"] = getpass.getpass("2")
os.environ["VECTARA_API_KEY"] = getpass.getpass("zqt_Y3kD9bueJq3QO5t_FISVQLmgTWMDhzgMgK9Isw")

1668875253··········
2··········
zqt_Y3kD9bueJq3QO5t_FISVQLmgTWMDhzgMgK9Isw··········


In [ ]:
vectorstore = Vectara(
                vectara_customer_id=1668875253,
                vectara_corpus_id=2,
                vectara_api_key="zqt_Y3kD9bueJq3QO5t_FISVQLmgTWMDhzgMgK9Isw"
            )

In [ ]:
def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: Define Basics of finance , Risk Management , and investmentment banking
Answer:  Finance refers to the study of how individuals, businesses, governments, and organizations allocate resources to satisfy their needs and wants. It involves the principles of accounting, economics, and law. Risk management is the process of identifying, assessing, and controlling potential risks that may affect the achievement of organizational goals. Investment banking involves helping clients raise capital by underwriting and selling securities, providing advice on mergers and acquisitions, and managing assets.



In [ ]:
def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt: Give me points of Risk in opertaions and types of  investment banking with basics of finance
Answer: 

Points of Risk in Operations:

* Production Risks (Weather, Climate Changes, Pests, Diseases, Technology, Genetics, Machinery Efficiency, Quality of Inputs, Fire, Wind, Theft, Other Casualties)
* Marketing Risks (Unanticipated Forces, Weather or Government Action, Changes in Output and Input Prices, Access to Markets)
* Financial Risks (Cost and Availability of Capital, Ability to Meet Cash Flow Needs, Ability to Grow Equity, Ability to Absorb Short-Term Financial Shocks)

Types of Investment Banking:

* Mergers and Acquisitions (M&A)
* Initial Public Offerings (IPOs)
* Debt Financing
* Equity Financing
* Private Placements
* Securities Underwriting

Basics of Finance:

* Assets: Anything of value that can be bought or sold, such as cash, inventory, property, or stocks.
* Liabilities: Obligations or debts that a company has, such as loans or accounts payable.
* Equity: Ownersh

In [ ]:
def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()


Prompt: Tell me the key points of budget speech andd credit risk
Answer:  The key points of the budget speech were:

* The government will introduce new tax laws to raise revenue
* The government will increase spending on infrastructure projects
* The government will create jobs in the manufacturing sector
* The government will encourage foreign investment in the country
* The government will provide subsidies to small businesses
* The government will increase funding for education and healthcare
* The government will reduce the deficit by cutting unnecessary spending
* The government will introduce a new fiscal policy to stimulate economic growth

Credit risk refers to the possibility that a borrower may default on a debt obligation. To manage credit risk, lenders typically use a variety of methods, including credit scoring, collateral requirements, and loan covenants. Credit scoring involves evaluating a borrower's creditworthiness based on factors such as their credit history, incom